In [14]:
from tools.tools import Reddit
from tools.translations import IO, POST_TYPE
import pandas as pd
from datetime import datetime as dt

In [2]:
# https://www.reddit.com/dev/api/
params = {
    "limit": "100",
    }

In [3]:
r = Reddit(params)

In [4]:
url = IO["BASE"] + IO["SUBR"]["STOCKMARKET"] + IO["LIST"]["HOT"]
print(url)

https://oauth.reddit.com/r/StockMarket/hot/


In [5]:
#print(r.pretty(r.open(url)))
#print(r.open(url))

In [23]:
df = pd.DataFrame()

for post in r.open(url)["data"]["children"]:
    award_names = {
        f"award_name{i}": award["name"]
        for i, award in enumerate(post["data"]["all_awardings"])
        }
    award_counts = {
        f"award_count{i}": award["count"]
        for i, award in enumerate(post["data"]["all_awardings"])
        }
    df = df.append({
        "id": post["kind"] + "_"+ post["data"]["id"],
        "created_at": dt.utcfromtimestamp(post["data"]["created_utc"]),
        "kind": POST_TYPE[post["kind"]],
        "subreddit": post["data"]["subreddit"],
        "title": post["data"]["title"],
        "permalink": post["data"]["permalink"],
        "num_comments": post["data"]["num_comments"],
        "score": post["data"]["score"],
        "ups": post["data"]["ups"],
        "downs": post["data"]["downs"],
        "upvote_ratio": post["data"]["upvote_ratio"],
        "selftext": post["data"]["selftext"],
        "total_awards_received": post["data"]["total_awards_received"],
        **award_names,
        **award_counts
        }, ignore_index=True)

In [24]:
df.head().style

In [ ]:
print(df)
print(IO["BASE"] + df.loc[0].permalink)
print(r.pretty(r.open(IO["BASE"] + df.loc[0].permalink)))

#0 = post
#1..n = comments
# data -> children
#name
#body
#archived
#controversiality
#created_utc
#depth
#downs
#link_id
#parent_id
#permalink
#kind
#replies <- recursion